# Apocalypse Now Chatbot

## Imports

In [14]:
import os
from dotenv import load_dotenv
import pandas as pd 
from anthropic import AnthropicVertex
import vertexai
import json


## Credentials

In [15]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcp-nyc-ce93903e254d.json'

PROJECT_ID = "gcp-nyc"
REGION = "us-central1"

In [16]:
embeddings_model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

embeddings = embeddings_model.get_embeddings(["Dinner in New York City"])
for embedding in embeddings:
  vector = embedding.values
  print(vector)

[0.03092864528298378, -0.029868636280298233, -0.029459740966558456, -0.0009445197647437453, 0.002139341551810503, 0.003000705735757947, 0.013629520311951637, 0.04873180389404297, 0.010373848490417004, 0.026051972061395645, 0.016042839735746384, -0.0191232617944479, -0.01023846585303545, -0.04757988452911377, -0.006645353976637125, 0.015495737083256245, -0.01545269787311554, 0.015853025019168854, 0.0457160621881485, -0.03305236995220184, -0.04611865431070328, 0.008199586533010006, -0.028266873210668564, -0.023114023730158806, -0.019202029332518578, 0.004720576573163271, 0.053389158099889755, -0.06605908274650574, -0.011999640613794327, 0.0033735090401023626, -0.04246285557746887, 0.04036254063248634, -0.0734783336520195, 0.041490502655506134, 0.035548970103263855, -0.04976203665137291, -0.0034386806655675173, 0.03629659116268158, -0.006216620095074177, 0.04435154050588608, -0.024378718808293343, -0.029397232457995415, -0.0809723362326622, -0.05687682703137398, 0.029550012201070786, -0.0

In [17]:
MODEL = "claude-3-sonnet@20240229"
client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for Pizza.",
        }
    ],
    model=MODEL,
)
data = json.loads(message.model_dump_json(indent=2))["content"][0]
print(data["text"])

Here is a classic recipe for making pizza at home:

Pizza Dough:
- 1 cup warm water (110°F/45°C)
- 1 teaspoon sugar
- 1 teaspoon salt
- 1 tablespoon olive oil
- 2 1/4 teaspoons active dry yeast (1 standard packet)
- 2 1/2 cups all-purpose flour 

Pizza Sauce:
- 1 can (15 oz) tomato sauce
- 1 teaspoon dried basil
- 1 teaspoon dried oregano
- 1/2 teaspoon garlic powder
- 1/4 teaspoon salt
- 1/4 teaspoon black pepper

Toppings:
- Shredded mozzarella cheese
- Grated parmesan cheese
- Sliced pepperoni or other desired toppings

Instructions:

1. Make the dough: In a large bowl, combine the warm water, sugar, salt, olive oil and yeast. Let sit for 5 minutes until foamy. Add the flour and stir until a shaggy dough forms. Turn out onto a floured surface and knead for 3-5 minutes until smooth and elastic. Place in a greased bowl, cover and let rise for 1 hour.

2. Make the sauce: In a small bowl, mix together the tomato sauce, basil, oregano, garlic powder, salt and pepper.

3. Preheat oven to 